### Importación de librerías

In [185]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [186]:
# cargar el archivo marketingcampaigns_corrected.csv (datos corregidos)
df = pd.read_csv('../data/marketingcampaigns_corrected.csv')
df

,campaign_name,start_date,end_date,budget,roi,type,target_audience,channel,conversion_rate,revenue
0,Public-key multi-tasking throughput,2023-04-01,2024-02-23,8082.3,0.35,email,B2B,organic,0.40,709593.48
1,De-engineered analyzing task-force,2023-02-15,2024-04-22,17712.98,0.74,email,B2C,promotion,0.66,516609.10
2,Balanced solution-oriented Local Area Network,2022-12-20,2023-10-11,84643.1,0.37,podcast,B2B,paid,0.28,458227.42
3,Distributed real-time methodology,2022-09-26,2023-09-27,14589.75,0.47,webinar,B2B,organic,0.19,89958.73
4,Front-line executive infrastructure,2023-07-07,2024-05-15,39291.9,0.30,social media,B2B,promotion,0.81,47511.35
...,...,...,...,...,...,...,...,...,...,...
1032,No revenue campaign,2023-02-01,2023-08-01,20000,0.30,social media,B2B,organic,0.50,NaN
1033,Random mess,2023-06-06,NaN,100000,NaN,podcast,NaN,referral,NaN,300000.00
1034,Invalid budget,2022-12-01,2023-06-01,abc,NaN,email,B2C,promotion,0.20,50000.00
1035,Overlapping dates,2023-03-01,2022-12-31,60000,0.60,webinar,B2B,paid,0.70,90000.00


In [187]:
# información preliminar de los datos
'''
campaign_name -----> nombre_de_campaña
start_date --------> fecha_de_inicio
end_date ----------> fecha_de_finalización
budget ------------> presupuesto
roi ---------------> ROI "retorno de la inversión" (Return On Investment)
type --------------> tipo
target_audience ---> público_objetivo
channel -----------> canal
conversion_rate ---> tasa_de_conversión
revenue -----------> ingresos
'''
df.info()
# OBSERVACIONES:
# start_date y end_date son de tipo object, se deben convertir a datetime
# conversion_rate es de tipo object, se debe convertir a float
# revenue es de tipo object, se debe convertir a float

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1037 entries, 0 to 1036
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   campaign_name    1037 non-null   object 
 1   start_date       1036 non-null   object 
 2   end_date         1035 non-null   object 
 3   budget           1034 non-null   object 
 4   roi              1033 non-null   float64
 5   type             1036 non-null   object 
 6   target_audience  1035 non-null   object 
 7   channel          1036 non-null   object 
 8   conversion_rate  1033 non-null   float64
 9   revenue          1034 non-null   float64
dtypes: float64(3), object(7)
memory usage: 81.1+ KB


In [188]:
df.describe()
# OBSERVACIONES:
# hay al menos un registro con valores negativos de roi y revenue
# hay valores demasiado separados (desviación estándar muy grande) en budget

,roi,conversion_rate,revenue
count,1033.000000,1033.000000,1034.000000
mean,0.533553,0.541946,509901.685667
std,0.261510,0.267271,287916.037107
min,-0.200000,0.000000,-15000.000000
25%,0.310000,0.300000,266545.580000
50%,0.530000,0.550000,516907.835000
75%,0.760000,0.770000,764547.970000
max,0.990000,1.500000,999712.490000


In [189]:
print(df.isnull().sum())
print(f"Suma de maximo de registros con nulos: {df.isnull().sum().sum()}")
# OBSERVACIONES:
# hay valores nulos a corregir en todas las columnas

campaign_name      0
start_date         1
end_date           2
budget             3
roi                4
type               1
target_audience    2
channel            1
conversion_rate    4
revenue            3
dtype: int64
Suma de maximo de registros con nulos: 21


In [ ]:
# convertir los datos de las columnas 'budget', 'roi', 'conversion_rate' y 'revenue' a numéricos, y reemplazar valores no numericos por NaN
for column in ['budget', 'roi', 'conversion_rate', 'revenue']:
  df[column] = pd.to_numeric(df[column], errors='coerce')
print(df.isnull().sum())

campaign_name      0
start_date         1
end_date           2
budget             4
roi                4
type               1
target_audience    2
channel            1
conversion_rate    4
revenue            3
dtype: int64


vemos que hay un registro adicional en el campo 'budget' que no es un número, por lo que se debe convertir a NaN para poder trabajar con el campo.

In [191]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1037 entries, 0 to 1036
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   campaign_name    1037 non-null   object 
 1   start_date       1036 non-null   object 
 2   end_date         1035 non-null   object 
 3   budget           1033 non-null   float64
 4   roi              1033 non-null   float64
 5   type             1036 non-null   object 
 6   target_audience  1035 non-null   object 
 7   channel          1036 non-null   object 
 8   conversion_rate  1033 non-null   float64
 9   revenue          1034 non-null   float64
dtypes: float64(4), object(6)
memory usage: 81.1+ KB


### PRIMERA LIMPIEZA DE NULOS
Se eliminarán los registros que contengan valores de nulos que, por su importancia o por su cantidad en el registro, hagan que el registro sea inútil para el análisis.

In [192]:
# nuevo campo que indica la cantidad de nulos en cada registro (este campo se eliminará más adelante)
df['num_nulls'] = df.isnull().sum(axis=1)

# Ordenar el DataFrame por la columna de número de nulos en orden descendente
df_sorted_by_nulls = df.sort_values(by='num_nulls', ascending=False)

# Mostrar los registros con nulos, ordenados por número de nulos en cada registro
df_sorted_by_nulls[df_sorted_by_nulls['num_nulls'] > 0]

,campaign_name,start_date,end_date,budget,roi,type,target_audience,channel,conversion_rate,revenue,num_nulls
1029,Null-heavy campaign,2023-01-01,NaN,NaN,NaN,B2B,social media,NaN,NaN,NaN,6
1033,Random mess,2023-06-06,NaN,100000.00,NaN,podcast,NaN,referral,NaN,300000.00,4
1026,Cloud-based scalable solution,NaN,2023-12-31,50000.00,NaN,event,B2C,paid,0.30,120000.00,2
1034,Invalid budget,2022-12-01,2023-06-01,NaN,NaN,email,B2C,promotion,0.20,50000.00,2
1008,NEW CAMPAIGN - Missing Budget,2023-10-01,2024-01-15,NaN,0.25,email,B2B,organic,NaN,45000.00,2
1027,Broken-date campaign,2023-13-01,2024-01-01,25000.00,0.45,email,B2B,organic,NaN,87500.00,1
1028,Negative ROI test,2022-10-10,2023-05-05,-10000.00,-0.20,podcast,B2C,referral,0.10,NaN,1
1005,Upgradable transitional data-warehouse,2023-06-29,2023-12-13,NaN,0.59,social media,B2C,referral,0.67,558302.11,1
1031,Extra long name campaign test,2023-04-15,2023-09-15,30000.00,0.25,email,NaN,paid,0.40,45000.00,1
1032,No revenue campaign,2023-02-01,2023-08-01,20000.00,0.30,social media,B2B,organic,0.50,NaN,1


In [ ]:
# eliminar registros con un número de nulos por registro mayor a 1
df = df[df['num_nulls'] <= 1]
df[df['num_nulls'] > 0]

,campaign_name,start_date,end_date,budget,roi,type,target_audience,channel,conversion_rate,revenue,num_nulls
1005,Upgradable transitional data-warehouse,2023-06-29,2023-12-13,NaN,0.59,social media,B2C,referral,0.67,558302.11,1
1007,User-friendly client-driven service-desk,2023-01-06,2023-12-11,36800.58,0.40,NaN,B2C,promotion,0.52,206241.46,1
1027,Broken-date campaign,2023-13-01,2024-01-01,25000.00,0.45,email,B2B,organic,NaN,87500.00,1
1028,Negative ROI test,2022-10-10,2023-05-05,-10000.00,-0.20,podcast,B2C,referral,0.10,NaN,1
1031,Extra long name campaign test,2023-04-15,2023-09-15,30000.00,0.25,email,NaN,paid,0.40,45000.00,1
1032,No revenue campaign,2023-02-01,2023-08-01,20000.00,0.30,social media,B2B,organic,0.50,NaN,1


In [ ]:
# dividir registros por tipo de dato object, y hallar la cantidad de valores únicos en cada columna para ver si hay errores
print(df.groupby('type').size())
print(f'Suma de registros: {df.groupby('type').size().sum()}\n')
print(df.groupby('channel').size())
print(f'Suma de registros: {df.groupby('channel').size().sum()}\n')
print(df.groupby('target_audience').size())
print(f'Suma de registros: {df.groupby('target_audience').size().sum()}\n')
print(f'Suma de registros totales: {df['campaign_name'].count()}')

# OBSERVACIONES:
# hay un valor incorrecto (referal) en la columna channel --> cambiar por referral
# el resto de valores son consistentes (no se detectan otros errores de escritura)

type
email           288
podcast         233
social media    242
webinar         268
dtype: int64
Suma de registros: 1031

channel
organic      249
paid         243
promotion    281
referal        1
referral     258
dtype: int64
Suma de registros: 1032

target_audience
B2B    530
B2C    501
dtype: int64
Suma de registros: 1031

Suma de registros totales: 1032


### CORRECCIÓN DE ERRORES A HACER
1. ESCRITURA DE DATOS
* hay un valor incorrecto (referal) en la columna channel --> cambiar por referral

2. NULOS
* hay valores nulos a corregir en las columnas: `budget`, `type`, `target_audience`, `conversion_rate` y `revenue`

3. TYPADO
* start_date y end_date son de tipo object, se deben convertir a datetime
* conversion_rate es de tipo object, se debe convertir a float
* revenue es de tipo object, se debe convertir a float

4. DISPERSIÓN
* hay al menos un registro con valores negativos de roi y revenue
* hay valores demasiado separados (desviación estándar muy grande) en budget

### 1. ESCRITURA DE DATOS

In [196]:
# cambiar el valor referal por referral en la columna channel
df.loc[df['channel'] == 'referal', 'channel'] = 'referral'
print(df.groupby('channel').size())
print(f'Suma de registros: {df.groupby('channel').size().sum()}\n')

channel
organic      249
paid         243
promotion    281
referral     259
dtype: int64
Suma de registros: 1032



### 2. NULOS

In [202]:
# suma de valores nulos en cada columna
df.isnull().sum()

campaign_name      0
start_date         0
end_date           0
budget             1
roi                0
type               0
target_audience    1
channel            0
conversion_rate    1
revenue            2
num_nulls          0
dtype: int64

#### 2.1. Columna `type`

In [198]:
# registro con valores nulos en la columna type
df[df['type'].isnull()]

,campaign_name,start_date,end_date,budget,roi,type,target_audience,channel,conversion_rate,revenue,num_nulls
1007,User-friendly client-driven service-desk,2023-01-06,2023-12-11,36800.58,0.4,NaN,B2C,promotion,0.52,206241.46,1


In [199]:
# contar registros con los valores channel=promotion y target_audence=B2C, agrupados por type
df[(df['channel'] == 'promotion') & (df['target_audience'] == 'B2C')].groupby('type').size()

type
email           37
podcast         30
social media    23
webinar         39
dtype: int64

In [200]:
# cambiar el valor nulo en la columna type por la mayor coincidencia con el resultado del filtro anterior (webinar)
df.loc[df['type'].isnull(), 'type'] = 'webinar'
df.isnull().sum()

campaign_name      0
start_date         0
end_date           0
budget             1
roi                0
type               0
target_audience    1
channel            0
conversion_rate    1
revenue            2
num_nulls          0
dtype: int64

#### 2.2. Columna `budget`
Correción de nulos en la columna `budget`.

In [201]:
# registro con valores nulos en la columna budget
df[df['budget'].isnull()]

,campaign_name,start_date,end_date,budget,roi,type,target_audience,channel,conversion_rate,revenue,num_nulls
1005,Upgradable transitional data-warehouse,2023-06-29,2023-12-13,NaN,0.59,social media,B2C,referral,0.67,558302.11,1


In [ ]:
print(f'Media de la columna \'budget\': {df['budget'].mean()}')
print(f'\nMedia de \'budget\': {df[(df['target_audience'] == 'B2C') &
                                    (df['type'] == 'social media')]['budget'].mean()}\nfiltros\n\ttype: social media\n\ttarget_audience: B2C\t')
print(f'\nMedia de \'budget\': {df[(df['target_audience'] == 'B2C') &
                                    (df['type'] == 'social media') &
                                      (df['channel'] == 'referral')]['budget'].mean()}\nfiltros\n\ttype: social media\n\ttarget_audience: B2C\n\tchannel: referral\t')

df[(df['target_audience'] == 'B2C') & (df['type'] == 'social media') & (df['channel'] == 'referral')].sort_values(by='revenue', ascending=False)

Media de la columna 'budget': 58798.43158098933

Media de 'budget': 45492.14392857142
filtros
	type: social media
	target_audience: B2C	

Media de 'budget': 42925.83965517241
filtros
	type: social media
	target_audience: B2C
	channel: referral	


,campaign_name,start_date,end_date,budget,roi,type,target_audience,channel,conversion_rate,revenue,num_nulls
373,Configurable client-driven migration,2022-12-09,2024-03-10,21172.32,0.16,social media,B2C,referral,0.32,972694.30,0
704,Innovative intangible complexity,2022-09-03,2024-03-04,19992.62,0.46,social media,B2C,referral,0.62,910901.40,0
466,Reduced content-based Graphic Interface,2022-10-26,2024-03-03,16945.29,0.72,social media,B2C,referral,0.76,884669.86,0
89,Stand-alone bandwidth-monitored capability,2022-10-10,2024-03-29,89383.19,0.83,social media,B2C,referral,0.77,828963.15,0
424,Mandatory 3rdgeneration matrices,2023-07-02,2023-10-28,29709.16,0.86,social media,B2C,referral,0.12,811850.37,0
169,Ergonomic next generation hub,2022-12-18,2024-01-27,27079.85,0.32,social media,B2C,referral,0.30,784636.20,0
976,User-centric clear-thinking interface,2022-08-28,2024-05-27,29206.80,0.60,social media,B2C,referral,0.34,628376.14,0
980,Reverse-engineered 4thgeneration analyzer,2022-12-24,2023-11-03,35539.13,0.53,social media,B2C,referral,0.68,579364.90,0
1023,Intuitive responsive support,2022-11-25,2024-04-04,1816.22,0.81,social media,B2C,referral,0.85,563280.30,0
9,Intuitive responsive support,2022-11-25,2024-04-04,1816.22,0.81,social media,B2C,referral,0.85,563280.30,0


comprobado que el valor faltante en la columna budget corresponde con un registro repetido, se elimina el registro.

In [217]:
# eliminación de registros con valores nulos en la columna budget
df = df.dropna(subset=['budget'])

In [218]:
df.isnull().sum()

campaign_name      0
start_date         0
end_date           0
budget             0
roi                0
type               0
target_audience    1
channel            0
conversion_rate    1
revenue            2
num_nulls          0
dtype: int64

In [219]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1031 entries, 0 to 1036
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   campaign_name    1031 non-null   object 
 1   start_date       1031 non-null   object 
 2   end_date         1031 non-null   object 
 3   budget           1031 non-null   float64
 4   roi              1031 non-null   float64
 5   type             1031 non-null   object 
 6   target_audience  1030 non-null   object 
 7   channel          1031 non-null   object 
 8   conversion_rate  1030 non-null   float64
 9   revenue          1029 non-null   float64
 10  num_nulls        1031 non-null   int64  
dtypes: float64(4), int64(1), object(6)
memory usage: 96.7+ KB


#### 2.3. Columna `target_audience`
Correción de nulos en la columna `target_audience`.

In [222]:
# registro con valores nulos en la columna type
df[df['target_audience'].isnull()]

,campaign_name,start_date,end_date,budget,roi,type,target_audience,channel,conversion_rate,revenue,num_nulls
1031,Extra long name campaign test,2023-04-15,2023-09-15,30000.0,0.25,email,NaN,paid,0.4,45000.0,1


In [ ]:
df[(df['type'] == 'email') & (df['channel'] == 'paid')].sort_values(by='revenue', ascending=False)

,campaign_name,start_date,end_date,budget,roi,type,target_audience,channel,conversion_rate,revenue,num_nulls
108,User-centric responsive software,2022-09-24,2023-08-04,55105.95,0.39,email,B2B,paid,0.65,980593.34,0
257,Self-enabling reciprocal algorithm,2022-11-01,2024-07-29,8645.67,0.12,email,B2B,paid,0.37,961551.85,0
635,Intuitive didactic interface,2022-09-06,2024-07-17,41350.29,0.82,email,B2C,paid,0.18,953178.97,0
32,Seamless zero-defect portal,2022-08-12,2024-07-19,71277.10,0.21,email,B2B,paid,0.56,932824.61,0
734,Grass-roots client-server middleware,2023-01-04,2023-11-10,41458.15,0.97,email,B2B,paid,0.21,924885.11,0
...,...,...,...,...,...,...,...,...,...,...,...
681,Organic actuating firmware,2023-07-07,2023-10-03,38665.10,0.14,email,B2B,paid,0.29,74042.78,0
458,Customizable global hierarchy,2023-04-26,2023-10-12,86716.38,0.74,email,B2B,paid,0.95,69360.86,0
1031,Extra long name campaign test,2023-04-15,2023-09-15,30000.00,0.25,email,NaN,paid,0.40,45000.00,1
862,Inverse system-worthy utilization,2022-08-25,2023-08-15,17398.10,0.22,email,B2C,paid,0.60,36397.45,0
